In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pyspark

from pyspark.sql import Row


import folium
import html
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from __future__ import division
import time

In [5]:
import pandas as pd

In [2]:
#path

data = '/Users/jerrygeorge/Desktop/finalproj/data/'
model = '/Users/jerrygeorge/Desktop/finalproj/model/'
output = '/Users/jerrygeorge/Desktop/finalproj/output/'

In [3]:
#load the dataset to spark

business_df = spark.read.parquet(data + 'business-small.parquet')
user_df = spark.read.parquet(data + 'user-small.parquet')
review_df = spark.read.parquet(data + 'review-small.parquet')

In [4]:
review=review_df.toPandas()
business=business_df.toPandas()

In [6]:
rest_t = business

# Join restaurants
review_tor = pd.merge(rest_t, review, on='business_id', how='left')

uL = review_tor[['user_id','business_id','stars_y']]

# Indexing
u_index = uL.user_id.unique()
i_index = uL.business_id.unique()

# Count unique users and restaurants
u_users = uL.user_id.unique().shape[0]
u_items = uL.business_id.unique().shape[0]

In [7]:
# Train Test 70/30
from sklearn.model_selection import train_test_split
train, test = train_test_split(uL, test_size=0.30, random_state=122)

tr_m = pd.DataFrame(index=u_index, columns=i_index)

for row in train.itertuples():
    user = row[1]
    item = row[2]
    tr_m.loc[user][item] = row[3]  

test_m = pd.DataFrame(index=u_index, columns=i_index)

for row in test.itertuples():
    user = row[1]
    item = row[2]
    test_m.loc[user][item] = row[3]

it1 = train_matrix.apply(lambda x: x > 0, raw=True).sum(axis=1)

# users with >5 items
train1 = tr_m
train1['item_1'] = it1
train2 = train1.loc[train1['item_1'] >= 5]

train2 = train2.drop('item_1',axis=1)
train3 = train2.transpose()
user_1 = train3.apply(lambda x: x > 0, raw=True).sum(axis=1)

# items with >5 items
train3['user_1'] = user_1
train4 = train3.loc[train3['user_1'] >= 5]
train4 = train4.drop('user_1',axis=1)
train5 = train4.transpose()

it2 = train5.apply(lambda x: x > 0, raw=True).sum(axis=1)

train5['item_2'] = it2
train6 = train5.loc[train5['item_2'] >= 5]
train6 = train6.drop('item_2',axis=1)
train7 = train6.transpose()
u2 = train7.apply(lambda x: x > 0, raw=True).sum(axis=1)
train7['user_2'] = u2
train8 = train7.loc[train7['user_2'] >= 5]
train8 = train8.drop('user_2',axis=1)
train9 = train8.transpose()

it3 = train9.apply(lambda x: x > 0, raw=True).sum(axis=1)
u3 = train9.apply(lambda x: x > 0, raw=True).sum(axis=1)

test9 = test_matrix.loc[train9.index,train9.columns]


In [8]:
def cb(train, test, sim, type='user', knn=5):

    train_0 = train.fillna(0)
    test_0 = test.fillna(0)

    
    if sim == 'cosine':
        udist = pdist(train_0, metric='cosine')
        idist = pdist(train_0.T, metric='cosine')

    elif sim == 'pearson':
        udist = pdist(train_0, metric='correlation')
        idist = pdist(train_0.T, metric='correlation')
        
    usim = 1 - udist
    isim = 1 - idsit
 
    mr = train.mean(axis=1)
    mur = pd.concat([mr] * len(train.columns), axis=1)
    mur.columns = train.columns
    
    
    mur1 = mur
    mur1[train_0 == 0] = 0
    
    # Normalize Ratings
    rd = train_0 - mur1
    
    # Create Dataframe
    murf = pd.concat([mr] * len(train.columns), axis=1)
    murf.columns = train.columns
    
    pred = pd.DataFrame(index=train.index, columns=train.columns)
    
    # User-User Collaborative Filtering
    if type == 'user':    
        usim = pd.DataFrame(usim, index=train.index, columns=train.index)
        
        if knn != 'all':
            usimm = usim.as_matrix()
        
            cols = test.columns
            test_rated = test.apply(lambda x: x > 0, raw=True).apply(lambda x: list(cols[x.values]), axis=1)
            
            for m in range(len(usim)):
                temp = usimm[m]
                temp = pd.DataFrame(temp,index=train.index, columns=['similarity'])
                temp['rank'] = temp['similarity'].rank(ascending=0)
                
                for n in range(len(test_rated[m])):
                    temp2 = rd[[test_rated[m][n]]]
                    temp2.columns = ['rating']
                    # Join similarity, rank, rating 
                    result = pd.concat([temp, temp2], axis=1)    
                    result2 = result[result['rating'] != 0]
                    result3 = result2.nsmallest(int(knn), 'rank')
                
                   #Divide weighted sum by sum of similarities
                    score = result3['similarity'].dot(result3['rating'])/result3['similarity'].sum()
                    mean = murf.loc[train.index[m]][test_rated[m][n]]
                
                    # Make a prediction
                    pred.loc[train.index[m]][test_rated[m][n]] = mean + score
                    
            
        elif knn == 'all':  
            
            nu = usim.dot(rd)
            
            ssu = usim.sum(axis=1)
            ssmu = pd.concat([ssu] * len(train.columns), axis=1)
            ssmu.columns = train.columns

           
            pred = murf + nu / ssmu
                
    # Item-Item Collaborative Filtering  
    elif type == 'item':
        
        isim = pd.DataFrame(isim, index=train.columns, columns=train.columns)
        
        if knn != 'all':
            isimm = isim.as_matrix()

            cols2 = test.T.columns
            test_rated2 = test.T.apply(lambda x: x > 0, raw=True).apply(lambda x: list(cols2[x.values]), axis=1)
        
            for m in range(len(isim)):
                temp = isimm[m]
                temp = pd.DataFrame(temp,index=train.columns, columns=['similarity'])
                temp['rank'] = temp['similarity'].rank(ascending=0)

                for n in range(len(test_rated2[m])):
                    temp2 = rd.T[[test_rated2[m][n]]]
                    temp2.columns = ['rating']
                    result = pd.concat([temp, temp2], axis=1)    
                    result2 = result[result['rating'] != 0]
                    result3 = result2.nsmallest(int(knn), 'rank')
                
                  #Divide Weighted sum by sum of similarity
                    score = result3['similarity'].dot(result3['rating'])/result3['similarity'].sum()
                    mean = murf.loc[test_rated2[m][n]][train.columns[m]]
                
                    pred.loc[test_rated2[m][n]][train.columns[m]] = mean + score
        
      
        elif knn == 'all':
            
            ni = rd.dot(item_sim)

            ssi = isim.sum(axis=1)
            ssi = pd.DataFrame(ssi, index=train.columns)
            ssmi = pd.concat([ssi.T] * len(train), axis=0)

            ssmi.index = train.index

            pred = murf + ni / ssmi
            
    pred_0 = pred.fillna(0)
    pred_0_mat = pred_0.as_matrix()            
    return pred_0_mat

In [9]:
test9_0 = test9.fillna(0)
test9_0_mat = test9_0.as_matrix()


In [30]:
st = time.time()
p5 = cb(train9, test9, sim='cosine', type='user', knn='5')
p10 = cb(train9, test9, sim='cosine', type='user', knn='10')
p30 = cb(train9, test9, sim='cosine', type='user', knn='30')
pall = cb(train9, test9, sim='cosine', type='user', knn='all')
pi5 = cb(train9, test9, sim='cosine', type='item', knn='5')
pi10 = cb(train9, test9, sim='cosine', type='item', knn='10')
pi30 = cb(train9, test9, sim='cosine', type='item', knn='30')
piall = cb(train9, test9, sim='cosine', type='item', knn='all')
print ('8 Runs of Collaborative Filtering for Restaurant in Toronto took %s seconds' % (time.time() - st))

/Users/jerrygeorge/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:76: RuntimeWarning: invalid value encountered in double_scalars
/Users/jerrygeorge/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:136: RuntimeWarning: invalid value encountered in double_scalars


8 Runs of Collaborative Filtering for Restaurant in Toronto took 1328.1972260475159 seconds


In [11]:
st = time.time()
puc = cb(train9, test9, sim='cosine', type='user', knn='all')
pup = collaborative_filtering(train9, test9, sim='pearson', type='user', knn='all')
ptc = collaborative_filtering(train9, test9, sim='cosine', type='item', knn='all')
ptp = collaborative_filtering(train9, test9, sim='pearson', type='item', knn='all')
print ('4 Runs of Collaborative Filtering for Restaurant in Toronto took %s seconds' % (time.time() - st))

4 Runs of Collaborative Filtering for Restaurant in Toronto took 1019.102697134018 seconds


In [10]:
from sklearn.metrics import mean_absolute_error
def mae(prediction, actual):
    prediction = prediction[actual.nonzero()].flatten() 
    actual = actual[actual.nonzero()].flatten()
    return mean_absolute_error(prediction, actual)

In [25]:
print ('User-based CF using Cosine Similarity MAE: ' + str(mae(puc, test9_0_mat)))
print ('User-based CF using Pearson Correlation MAE: ' + str(mae(pup, test9_0_mat)))
print ('Item-based CF using Cosine Similarity MAE: ' + str(mae(pic, test9_0_mat)))
print ('Item-based CF using Pearson Correlation MAE: ' + str(mae(pip, test9_0_mat)))

User-based CF using Cosine Similarity MAE: 0.870475466574
User-based CF using Pearson Correlation MAE: 0.88131123966
Item-based CF using Cosine Similarity MAE: 0.872175443685
Item-based CF using Pearson Correlation MAE: 0.872637285628
